In [1]:
import numpy as np
import pandas as pd
data = pd.read_csv("input_datasets/help_a.csv")

# data['entrees_fr'] = data['entrees_fr'].replace('[$\s]', '', regex=True).replace('-', np.nan, regex=True).astype(float)
# data['entrees_usa'] = data['entrees_usa'].replace('[$\s]', '', regex=True).replace('-', np.nan, regex=True).astype(float)
df = pd.DataFrame(data)

data_jp = data
data_jp.fillna(-1, inplace=True)
display(data.head())


,titre,acteurs,budget,compositeur,date_sortie_fr,date_sortie_us,duree,entrees_fr,franchise,genres,pays,pegi_fr,pegi_us,producteur,realisateur,entrees_usa,salles_fr,studio
0,1492 christophe colomb,"gérard depardieu, sigourney weaver",47000000,pas de compositeur,1992-09-28,1992-10-09,162,610299.0,0,aventure - action,grande-bretagne,tous publics,PG,ridley scott,ridley scott,3002680.0,-1,gaumont
1,2 fast 2 furious,"eva mendes, paul walker",76000000,pas de compositeur,2003-06-18,2003-06-06,105,804821.0,1,aventure - action,etats-unis,tous publics,PG,neal h. moritz,john singleton,50472480.0,448,uip
2,40 ans : mode d'emploi,"paul rudd, melissa mccarthy, jason segel",35000000,pas de compositeur,2013-03-13,2012-12-21,133,125161.0,0,comédie,etats-unis,tous publics,R,judd apatow,judd apatow,11579175.0,180,universal
3,"40 ans, toujours puceau","steve carell, paul rudd, seth rogen, kevin har...",26000000,pas de compositeur,2005-11-09,2005-08-19,116,205065.0,0,comédie,etats-unis,tous publics,R,judd apatow,judd apatow,21422815.0,299,uip
4,58 minutes pour vivre,bruce willis,70000000,pas de compositeur,1990-10-03,1990-07-06,123,343054.0,1,aventure - action,etats-unis,-1,R,joel silver,renny harlin,21744661.0,-1,20th century fox


In [2]:
data_jp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   titre           365 non-null    object 
 1   acteurs         365 non-null    object 
 2   budget          365 non-null    int64  
 3   compositeur     365 non-null    object 
 4   date_sortie_fr  365 non-null    object 
 5   date_sortie_us  365 non-null    object 
 6   duree           365 non-null    int64  
 7   entrees_fr      365 non-null    float64
 8   franchise       365 non-null    int64  
 9   genres          365 non-null    object 
 10  pays            365 non-null    object 
 11  pegi_fr         365 non-null    object 
 12  pegi_us         365 non-null    object 
 13  producteur      365 non-null    object 
 14  realisateur     365 non-null    object 
 15  entrees_usa     365 non-null    float64
 16  salles_fr       365 non-null    int64  
 17  studio          365 non-null    obj

In [14]:
# data_jp["budget"].value_counts()
display(data_jp['pays'].unique())


array(['grande-bretagne', 'etats-unis', 'france', 'japon'], dtype=object)